# 5.11. Klasyfikacja za pomocą SVM

Powróćmy po raz kolejny do klasyfikacji niewypłacalności klientów. Spróbujemy, tym razem za pomocą SVM, rozwiązać kolejny raz ten sam problem. Jest to dość typowe zjawisko w przypadku tworzenia rozwiązań ML - należy stworzyć kilka modeli, aby później wybrać ten najlepszy, względnie wystarczająco dobry, ale wydajny. Poprzednio testowaliśmy KNN i jego wydajność dość mocno zależy od tego na jak wielu wektorach trenowaliśmy nasz model, co czyni go mało użytecznym gdy danych jest naprawdę sporo.

In [1]:
import pandas as pd

In [2]:
credit_cards_df = pd.read_parquet("../data/credit-cards-reduced.parquet")
credit_cards_df.sample(n=5).T

ID,8223,3151,5714,4793,24536
AGE,37.000000,52.0,22.000000,44.000000,39.000000
PAY_1,-2.000000,-2.0,3.000000,0.000000,0.000000
PAY_2,-2.000000,-2.0,2.000000,0.000000,0.000000
PAY_3,-2.000000,-2.0,0.000000,0.000000,0.000000
PAY_4,-2.000000,-2.0,0.000000,0.000000,0.000000
PAY_5,-2.000000,-2.0,2.000000,0.000000,0.000000
PAY_6,-1.000000,-2.0,2.000000,0.000000,2.000000
PAY_OVERDUE_COUNT,0.000000,0.0,4.000000,0.000000,1.000000
WEIGHTED_PAYMENT_HISTORY,-4.733333,-4.9,4.733333,0.000000,0.333333
AVG_PAY_AMT,3674.500000,0.0,1535.833333,1217.666667,2045.833333


Celowo nie ładujemy pełnego zbioru, żeby być w stanie porównać ze sobą wszystkie stworzone dotychczas rozwiązania. Regresja logistyczna działała najlepiej właśnie na tym podzbiorze i niekoniecznie oznacza to, że inne metody nie będą również w stanie wyciągnąć żadnych wniosków z pozostałych atrybutów. Wybór najlepszego zestawu cech jest dość specyficzny dla każdego modelu z osobna - to że w naszym przypadku regresja logistyczna nie dała rady z pewnych informacji skorzystać, nie może być podstawą do stwierdzenia, że nie mają one wartości.

In [3]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test = train_test_split(credit_cards_df,
                                   test_size=0.2, 
                                   random_state=2020)

In [5]:
svm = LinearSVC()

Nie popełnijmy znowu tego samego błędu jak przy KNN i zeskalujmy wszystkie zmienne, żeby uniknąć problemów z obniżoną skutecznością. Co do zasady, modele liniowe raczej wymagają skalowania zmiennych i warto jest zawsze uwzględnić to w procesie modelowania.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [7]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("classifier", LinearSVC())
])

In [8]:
pipeline.fit(X_train.drop(columns="DEFAULT"), 
             X_train["DEFAULT"])

C:\Users\lukaw\anaconda3\envs\intro-to-ml\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=None,
                           tol=0.0001, verbose=0))],
         verbose=False)

In [9]:
from sklearn.metrics import f1_score

In [10]:
f1_score(X_test["DEFAULT"],
         pipeline.predict(X_test.drop(columns="DEFAULT")))

0.416796267496112

In [11]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("classifier", LinearSVC(max_iter=10000))
])
pipeline.fit(X_train.drop(columns="DEFAULT"), 
             X_train["DEFAULT"])

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=10000,
                           multi_class='ovr', penalty='l2', random_state=None,
                           tol=0.0001, verbose=0))],
         verbose=False)

In [12]:
f1_score(X_test["DEFAULT"],
         pipeline.predict(X_test.drop(columns="DEFAULT")))

0.41597510373443985

## Nieliniowe rozszerzenia SVM

Sprawdźmy może, jak dla naszego problemu zadziałają nieliniowe wersje SVM. Dogłębna analiza tych metod to temat bardzo szeroki, a sam SVM jest w stanie rozwiązać naprawdę szeroką gamę problemów.

In [13]:
from sklearn.svm import SVC

In [14]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("classifier", SVC(kernel="poly"))
])
pipeline.fit(X_train.drop(columns="DEFAULT"), 
             X_train["DEFAULT"])
f1_score(X_test["DEFAULT"],
         pipeline.predict(X_test.drop(columns="DEFAULT")))

0.3833865814696486

In [15]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("classifier", SVC(kernel="poly", degree=7))
])
pipeline.fit(X_train.drop(columns="DEFAULT"), 
             X_train["DEFAULT"])
f1_score(X_test["DEFAULT"],
         pipeline.predict(X_test.drop(columns="DEFAULT")))

0.3625541125541126

In [16]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("classifier", SVC(kernel="rbf"))
])
pipeline.fit(X_train.drop(columns="DEFAULT"), 
             X_train["DEFAULT"])
f1_score(X_test["DEFAULT"],
         pipeline.predict(X_test.drop(columns="DEFAULT")))

0.4747919725893294